## RAG

**Librerías necesarias:**

In [37]:
!pip install transformers

In [1]:
!pip install transformers faiss-cpu sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


**Paso 1: Preparar el dataset**

Crea una base de conocimiento estructurada en un archivo `JSON` o `CSV`. Ejemplo:

In [2]:
import pandas as pd

data = [
    {"title": "Python Basics", "content": "Python is a versatile programming language."},
    {"title": "Google Colab", "content": "Google Colab is a free platform for Python coding."},
    {"title": "RAG", "content": "RAG combines retrieval with generative models for context-based answers."}
]
df = pd.DataFrame(data)
df.to_csv("knowledge_base.csv", index=False)


**Paso 2: Índice de recuperación con embeddings**

Utilizaremos `sentence-transformers` para crear embeddings y construir un índice.

In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd

# Cargar dataset
df = pd.read_csv("knowledge_base.csv")

# Crear embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # Modelo rápido y eficiente
embeddings = model.encode(df['content'].tolist())

# Crear índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print(f"Documentos indexados: {index.ntotal}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Documentos indexados: 3


**Paso 3: Consulta al índice**

Busca los documentos más relevantes para una consulta.

In [4]:
def search(query, top_k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = [df.iloc[idx]['content'] for idx in indices[0]]
    return results

query = "¿Qué es RAG?"
top_docs = search(query)
print("Documentos relevantes:")
for doc in top_docs:
    print("-", doc)


Documentos relevantes:
- RAG combines retrieval with generative models for context-based answers.
- Python is a versatile programming language.
- Google Colab is a free platform for Python coding.


**Paso 4: Generación con un modelo preentrenado**

Usaremos `transformers` para cargar un modelo generador como `Flan-T5`.

In [5]:
from transformers import pipeline

# Cargar modelo generativo
generator = pipeline('text2text-generation', model='google/flan-t5-small')

# Combinar recuperación y generación
def generate_answer(query):
    retrieved_docs = search(query)
    context = " ".join(retrieved_docs)
    prompt = f"Contexto: {context}\nPregunta: {query}\nRespuesta:"
    result = generator(prompt, max_length=100, do_sample=True)
    return result[0]['generated_text']

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Respuesta generada: Python


In [7]:
query = "¿Qué es Google Colab?"
answer = generate_answer(query)
print("Respuesta generada:", answer)

Respuesta generada: Python is a versatile programming language


In [8]:
query = "¿Qué es Google Colab?"
answer = generate_answer(query)
print("Respuesta generada:", answer)

Respuesta generada: RAG combines retrieval with generative models for context-based answers


## EJEMPLO CON DATA DE CANDIDATOS

**Librerías necesarias:**

Primero, instala las librerías necesarias para la extracción de texto y procesamiento.

In [10]:
!pip install PyPDF2 sentence-transformers faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


**Extraer texto de los PDFs**

Usaremos PyPDF2 para extraer texto.

In [20]:
import os
import pandas as pd
from PyPDF2 import PdfReader

# Ruta a la carpeta con los PDFs
pdf_folder = "./data/"

# Extraer texto de los PDFs
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Procesar los PDFs en la carpeta
pdf_data = []
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        filepath = os.path.join(pdf_folder, filename)
        pdf_data.append({"file_name": filename, "content": extract_text_from_pdf(filepath)})

In [25]:
# Crear dataframe
df_pdfs = pd.DataFrame(pdf_data)
print(df_pdfs.head())  # Ver las primeras filas

                 file_name                                            content
0    Plan_Daniel_Noboa.pdf    \n1 \n \n  \n \n \n \n \n \n \n \n \n \nPLAN...
1  Plan_Luisa_Gonzalez.pdf   \n \n \n \n \n \n \n \n  \n \nPLAN DE TRABAJO...


**Preprocesar los textos**

Dividimos el texto en fragmentos (chunks) para facilitar la recuperación. Esto es útil si los documentos son largos.

In [26]:
import re

# Función para dividir el texto en fragmentos
def split_into_chunks(text, chunk_size=300):
    sentences = re.split(r'\.|\n', text)  # Dividir en oraciones o líneas
    chunks = []
    current_chunk = []
    current_size = 0

    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
        if current_size + len(sentence.split()) > chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_size = 0
        current_chunk.append(sentence)
        current_size += len(sentence.split())

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [29]:
# Aplicar la división en fragmentos
chunk_data = []
for _, row in df_pdfs.iterrows():
    chunks = split_into_chunks(row["content"])
    for chunk in chunks:
        chunk_data.append({"file_name": row["file_name"], "content": chunk})

# Crear dataframe de fragmentos
df_chunks = pd.DataFrame(chunk_data)
print(df_chunks.head())  # Ver las primeras filas

               file_name                                            content
0  Plan_Daniel_Noboa.pdf  1 PLAN DE TRABAJO PLURIANUAL PARA PRESIDENTE Y...
1  Plan_Daniel_Noboa.pdf  4 PRODUCTIVO Y MEDIO AMBIENTAL 66 Sectores est...
2  Plan_Daniel_Noboa.pdf  relevante es la falta de educación y las oport...
3  Plan_Daniel_Noboa.pdf  En contraste, países como Perú, Colombia, Chil...
4  Plan_Daniel_Noboa.pdf  seguro y próspero para todos los ecuator ianos...


**Crear el índice con embeddings**

Usaremos sentence-transformers para generar embeddings y FAISS para construir un índice de recuperación.

In [40]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Crear embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df_chunks['content'].tolist())

# Construir índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Agregar embeddings al dataframe
df_chunks['embedding'] = list(embeddings)
print(df_chunks.head())  # Visualizar fragmentos con embeddings

               file_name                                            content  \
0  Plan_Daniel_Noboa.pdf  1 PLAN DE TRABAJO PLURIANUAL PARA PRESIDENTE Y...   
1  Plan_Daniel_Noboa.pdf  4 PRODUCTIVO Y MEDIO AMBIENTAL 66 Sectores est...   
2  Plan_Daniel_Noboa.pdf  relevante es la falta de educación y las oport...   
3  Plan_Daniel_Noboa.pdf  En contraste, países como Perú, Colombia, Chil...   
4  Plan_Daniel_Noboa.pdf  seguro y próspero para todos los ecuator ianos...   

                                           embedding  
0  [0.03846974, -0.00090473075, -0.012624212, -0....  
1  [0.07702333, -0.0018847033, -0.013918393, -0.0...  
2  [9.5412855e-05, 0.033709496, -0.06853278, -0.0...  
3  [0.025044395, -0.11453657, -0.034659408, -0.04...  
4  [0.044258013, 0.019030705, -0.045455076, -0.09...  


**Consultar al índice**

Ahora, podemos buscar los fragmentos más relevantes para una consulta.

In [41]:
# Buscar en el índice
def search(query, top_k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = df_chunks.iloc[indices[0]].copy()  # Recuperar fragmentos relevantes
    results['distance'] = distances[0]          # Agregar las distancias
    return results

# Ejemplo de consulta
query = "¿Cuál es la posición de Luisa González con respecto a la seguridad?"
results = search(query)

# Visualizar resultados
print(results[['file_name', 'content', 'distance']])

                   file_name  \
70   Plan_Luisa_Gonzalez.pdf   
6      Plan_Daniel_Noboa.pdf   
113  Plan_Luisa_Gonzalez.pdf   

                                               content  distance  
70   PLAN DE TRABAJO DEL BINOMIO   PRESIDENCIAL DE ...  0.888646  
6    inestable: por eso, ha sido elemental la elabo...  0.937141  
113  En este plan de gobierno llamamos a la acción ...  0.946592  


**Generar respuestas con el modelo generativo**

Usamos Flan-T5 para crear respuestas basadas en el contexto recuperado.

In [42]:
from transformers import pipeline

# Cargar modelo generativo
generator = pipeline('text2text-generation', model='google/flan-t5-small')

# Combinar recuperación y generación
def generate_answer(query, top_k=3):
    retrieved_docs = search(query, top_k=top_k)
    context = " ".join(retrieved_docs['content'].tolist())
    prompt = f"Contexto: {context}\nPregunta: {query}\nRespuesta:"
    result = generator(prompt, max_length=200, do_sample=True)
    return result[0]['generated_text']

# Ejemplo de generación
query = "¿Cuál es la posición de Daniel Noboa con respecto al empleo?"
answer = generate_answer(query)
print("Respuesta generada:", answer)

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (2303 > 512). Running this sequence through the model will result in indexing errors


Respuesta generada: Observera a los programas de clásico y o intervenco que hace el doero de las naciones Rechazamos cualquier forma de dominaión, colonialismo or intervencionismo que socave las plataformas noo oponemos a la imposición de polticas y decisiones unilaterales que afectan negativamente a los pases y sus ciudadanos Défense los derechos de la pobreza y de la amenaza a la prosperidad y la autodeterminación de las ciudadanas noo oponemos a la imposición depolticas y decisiones unilaterales que afecten negativamente a los pases y sus


**Personalización de preguntas**

Puedes ajustar las preguntas para incluir comparaciones y temas específicos. Por ejemplo:

In [49]:
query = "¿Cuál es la posición de Luisa González respecto al empleo?"
answer = generate_answer(query)
print("Respuesta generada:", answer)

Respuesta generada: Aqu está y confree aqu en Ecuador , por lo que tomar medidas radicales per el que tomar medidas radicales per el mundo trazada, la participación de colectivos, sociedad civil, pueblos andnacionalidades pervado, por lo que participarán de fundamental apoyo para la transformación de los problemas sociales a soluciones ciudadanas
